In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.applications import ResNet50
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import cv2 
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import seaborn as sns
from tqdm import tqdm
import gc

In [ ]:
# Veri yolu
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train/'
val_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test'
# Sınıf isimleri
class_names = ['NORMAL', 'PNEUMONIA']

# Sınıf dağılımını sayma
train_counts = {class_name: len(os.listdir(os.path.join(train_dir, class_name))) for class_name in class_names}
val_counts = {class_name: len(os.listdir(os.path.join(val_dir, class_name))) for class_name in class_names}
test_counts = {class_name: len(os.listdir(os.path.join(test_dir, class_name)))for class_name in class_names}
# Sonuçları yazdırma
print("Eğitim seti sınıf dağılımı:", train_counts)
print("Doğrulama seti sınıf dağılımı:", val_counts)
print("Test seti sınıf dağılımı:", test_counts)

# Görselleştirme
plt.bar(train_counts.keys(), train_counts.values(), color=['blue', 'orange'])
plt.title('Eğitim Seti Sınıf Dağılımı')
plt.ylabel('Örnek Sayısı')
plt.show()


In [ ]:
import os
import cv2
import random
import matplotlib.pyplot as plt

# Görüntüleri yüklemek için fonksiyon
def load_random_images_from_class(directory, class_name, num_images=2, target_size=(224, 224)):
    class_dir = os.path.join(directory, class_name)
    image_files = random.sample(os.listdir(class_dir), num_images)  # Rastgele görüntüler seç
    
    images = []
    for img_file in image_files:
        img_path = os.path.join(class_dir, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (target_size[1], target_size[0]))  # Boyutlandır
        
        # Eğer gri tonlamalı (tek kanallı) bir görüntü ise, RGB formatına çevir
        if img.ndim == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        images.append(img)
    return images

# Normal sınıfından iki görüntü, Pneumonia sınıfından iki görüntü seçelim
normal_images = load_random_images_from_class(train_dir, 'NORMAL', num_images=2)
pneumonia_images = load_random_images_from_class(train_dir, 'PNEUMONIA', num_images=2)

# Seçilen görüntüleri yan yana gösterme
fig, axes = plt.subplots(1, 4, figsize=(15, 5))

for i, img in enumerate(normal_images + pneumonia_images):
    axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[i].axis('off')
    if i < 2:
        axes[i].set_title('NORMAL')
    else:
        axes[i].set_title('PNEUMONIA')
plt.savefig('ornek2.png')
plt.show()


In [ ]:

# Sınıf isimleri
class_names = ['NORMAL', 'PNEUMONIA']

# Veri yükleme fonksiyonu
def load_data_from_directory(directory, target_size=(224, 224)):
    images = []
    labels = []

    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        
        # Dosyaları döngüye almak için tqdm kullanma
        for filename in tqdm(os.listdir(class_dir), desc=f'Loading {class_name}', unit='file'):
            if filename.lower().endswith(('.jpeg', '.jpg', '.png')):  # Sadece resim dosyalarını al
                img_path = os.path.join(class_dir, filename)
                img = io.imread(img_path)  # Görüntüyü oku
                
                # Görüntü boyutunu kontrol et
                if img.shape[0] < target_size[0] or img.shape[1] < target_size[1]:
                    #print(f"Boyut küçük, atlanıyor: {img.shape} - Dosya: {img_path}")
                    continue  # Boyut küçükse geç
                
                # Görüntüyü yeniden boyutlandırma
                img = cv2.resize(img, (target_size[1], target_size[0]))  # OpenCV'de (y, x) şeklinde belirtilir

                # Eğer görüntü tek kanallı ise, 3 kanallı hale getir
                if img.ndim == 2:  # Gri tonlamalı (1 kanal)
                    img = np.stack((img,)*3, axis=-1)
                
                # RGB görüntü kontrolü
                if img.shape == (target_size[0], target_size[1], 3):  # RGB görüntü
                    images.append(img)
                    labels.append(label)
                else:
                    print(f"Uygun olmayan boyut: {img.shape} - Dosya: {img_path}")

    return np.array(images), np.array(labels)

# Eğitim, doğrulama ve test verilerini yükleme
train_images, train_labels = load_data_from_directory(train_dir)
val_images, val_labels = load_data_from_directory(val_dir)
test_images, test_labels = load_data_from_directory(test_dir)

# Boyutları kontrol et
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Val images shape:", val_images.shape)
print("Val labels shape:", val_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)


In [ ]:


# Data augmentation tanımlama
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Sınıf dağılımı fonksiyonu
def get_class_distribution(images, labels):
    return {
        0: len(images[labels == 0]),  # NORMAL
        1: len(images[labels == 1])   # PNEUMONIA
    }

# Verileri birleştirirken sadece gerekli miktarda sınıf verilerini al
def balanced_data_merge(train_images, val_images, test_images, train_labels, val_labels, test_labels):
    # Tüm verileri birleştirme
    all_images = np.concatenate((train_images, val_images, test_images), axis=0)
    all_labels = np.concatenate((train_labels, val_labels, test_labels), axis=0)

    # Bellek temizliği
    del train_images, val_images, test_images
    del train_labels, val_labels, test_labels
    gc.collect()

    # Sınıf dağılımını kontrol etme
    class_distribution = get_class_distribution(all_images, all_labels)
    avg_class_size = (class_distribution[0] + class_distribution[1]) / 2  # Ortalama sınıf sayısı

    # PNEUMONIA sınıfında fazla veri varsa, fazla veriyi çıkaralım
    if class_distribution[1] > avg_class_size:
        pneumonia_images = all_images[all_labels == 1]  # PNEUMONIA görüntüleri
        pneumonia_labels = all_labels[all_labels == 1]

        # Fazla veriyi rastgele çıkarma
        num_to_remove = class_distribution[1] - avg_class_size
        indices_to_remove = np.random.choice(len(pneumonia_images), int(num_to_remove), replace=False)
        pneumonia_images = np.delete(pneumonia_images, indices_to_remove, axis=0)
        pneumonia_labels = np.delete(pneumonia_labels, indices_to_remove, axis=0)

        # Güncellenmiş PNEUMONIA ve NORMAL verilerini birleştirme
        normal_images = all_images[all_labels == 0]  # NORMAL görüntüleri
        all_images = np.concatenate((normal_images, pneumonia_images), axis=0)
        all_labels = np.concatenate((np.zeros(len(normal_images)), pneumonia_labels), axis=0)

    return all_images, all_labels

# Verileri birleştirme ve dengeleme
all_images, all_labels = balanced_data_merge(train_images, val_images, test_images, train_labels, val_labels, test_labels)
del train_images, val_images, test_images
del train_labels, val_labels, test_labels

In [ ]:
# NORMAL sınıfındaki eksik verileri artırma
class_distribution = get_class_distribution(all_images, all_labels)
num_to_augment = int((class_distribution[1] + class_distribution[0]) / 2 - class_distribution[0])

if num_to_augment > 0:
    normal_images = all_images[all_labels == 0]  # NORMAL görüntüleri

    # Data augmentation uygulama
    augmented_images = []
    for img in normal_images:
        img = img.reshape((1,) + img.shape)  # (1, yükseklik, genişlik, kanallar)
        for batch in data_gen.flow(img, batch_size=1):
            augmented_images.append(batch[0].astype(np.uint8))
            if len(augmented_images) >= num_to_augment:
                break

    # Artırılmış verileri ekleme
    augmented_images = np.array(augmented_images)
    all_images = np.concatenate((all_images, augmented_images), axis=0)
    all_labels = np.concatenate((all_labels, np.zeros(len(augmented_images))), axis=0)

# Verileri X, y şeklinde ayırma
X = all_images
y = all_labels

# Bellek temizliği
del all_images
del all_labels


In [ ]:
# X ve y'yi dosyaya kaydetme
np.save('X_data.npy', X)  # X verilerini kaydet
np.save('y_data.npy', y)  

In [ ]:
del X
del y
# Verileri yükleme
X_loaded = np.load('/kaggle/working/X_data.npy')  # X verilerini yükle
y_loaded = np.load('/kaggle/working/y_data.npy')  # y etiketlerini yükle

print("Yüklenen X şekli:", X_loaded.shape)
print("Yüklenen y şekli:", y_loaded.shape)


In [ ]:
y.astype(int)
# Eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Eğitim seti boyutu:", X_train.shape, y_train.shape)
print("Test seti boyutu:", X_test.shape, y_test.shape)

# Verilerin dengesizliğini kontrol etme
unique, counts = np.unique(y_train, return_counts=True)
print("Eğitim seti sınıf dağılımı:", dict(zip(unique, counts)))

del X
del y

In [ ]:


# Manuel olarak indirilen dosya yolu
local_weights_path = '/kaggle/input/resnet50_weights_tf_dim_ordering_tf_kernels_notop/tensorflow2/default/1/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# ResNet50 modelini indirilen ağırlıklarla yükleme
base_model = ResNet50(weights=local_weights_path, include_top=False, input_shape=(224, 224, 3))

# Katmanları dondurma
for layer in base_model.layers[:-4]:  # Son 4 katmanı dondurmadan bırakıyoruz
    layer.trainable = False

# Modelin son katmanlarını değiştirelim
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),  
    Dropout(0.5), 
    Dense(1, activation='sigmoid')  # İkili sınıflandırma için sigmoid aktivasyonu kullanıyoruz
])

# Modelin eğitim parametrelerini ayarlama
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks tanımlama
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint('/best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

# Modelin eğitilmesi
history = model.fit(
    X_train,  # Eğitim verileri
    y_train,  # Eğitim etiketleri
    epochs=10,     # Eğitim dönemi sayısı
    validation_split=0.2,  # Doğrulama verileri
    batch_size=32,  # Batch boyutu
    callbacks=[reduce_lr, early_stopping, model_checkpoint]  # Callbacks ekleme
)


In [ ]:
from keras.models import load_model

# Modeli kaydetme
model.save('my_model.keras')  


In [ ]:
# Eğitim geçmişinin görselleştirilmesi
plt.figure(figsize=(12, 4))

# Loss grafiği
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Model Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kaybı')
plt.legend()

# Doğruluk grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Model Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# ROC Eğrisi ve AUC Hesaplama
y_pred_proba = model.predict(X_test).ravel()  # Test verileri üzerindeki tahminler
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# ROC Eğrisinin Görselleştirilmesi
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC Eğrisi (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Karışıklık Matrisi Hesaplama
y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Tahminler
conf_matrix = confusion_matrix(y_test, y_pred)

# Karışıklık Matrisinin Görselleştirilmesi
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['NORMAL', 'PNEUMONIA'], 
            yticklabels=['NORMAL', 'PNEUMONIA'])
plt.ylabel('Gerçek Etiketler')
plt.xlabel('Tahmin Edilen Etiketler')
plt.title('Karışıklık Matrisi')
plt.show()


In [ ]:
# Sınıflandırma Raporu
report = classification_report(y_test, y_pred, target_names=['NORMAL', 'PNEUMONIA'])
print("Sınıflandırma Raporu:\n", report)
